<a href="https://colab.research.google.com/github/60hr00t/draft1/blob/main/sample_chatBot2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets accelerate bitsandbytes sentencepiece fastapi uvicorn pyngrok nest_asyncio streamlit requests gTTS fuzzywuzzy python-Levenshtein streamlit-image-select streamlit-calendar pandas streamlit-mic-recorder openai-whisper


In [ ]:
# Detecting CUDA GPU with PyTorch
import torch, platform

if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

In [ ]:
# ==========================================================
# Upload a CSV with “prompt” and “response” columns in Colab
# and set `CSV_PATH` to the selected file.
# ==========================================================

import io, pandas as pd
from google.colab import files

print("Upload a CSV with columns: prompt,response")
uploaded = files.upload()
if uploaded:
    fname = list(uploaded.keys())[0]
    CSV_PATH = f"/content/{fname}"
    print("Using uploaded file:", CSV_PATH)


Upload a CSV with columns: prompt,response


Saving Vet.csv - Vet.csv to Vet.csv - Vet (2).csv
Using uploaded file: /content/Vet.csv - Vet (2).csv


In [ ]:
# ==================================================
# Fuzzy match answers from CSV and render as plain
# text or option buttons
# ==================================================
from fuzzywuzzy import fuzz
df = pd.read_csv(CSV_PATH)
required = {"prompt", "response"}
assert required.issubset(df.columns), f"CSV must include: {required}. Found: {df.columns.tolist()}"
print(f"Loaded {len(df)} rows.")

# This function returns the best-matching response string for user_msg
# or None if the best score is below threshold.
def answer_from_csv(user_msg, df, threshold=90):
    best_score = 0
    best_response = None
    for _, row in df.iterrows():
        score = fuzz.partial_ratio(user_msg.lower(), row["prompt"].lower())
        if score > best_score:
            best_score = score
            best_response = row["response"]
    return best_response if best_score >= threshold else None
     # Check if response contains semicolons
    if ";" in response:
        options = [opt.strip() for opt in response.split(";")]
        st.markdown(f"**{prompt.capitalize()} Options:**")
    for i, option in enumerate(options):
            if st.button(option, key=f"{prompt}_{i}"):
                st.chat_message("user").markdown(option)
                reply = get_bot_response(option)  # Send to your backend
                st.chat_message("assistant").markdown(reply)
    else:
# If no semicolon, show as regular response
        st.markdown(f"**{prompt.capitalize()}:** {response}")




Loaded 2 rows.


In [ ]:
# =====================================================================
# This snippet downloads and initializes Google’s FLAN-T5 (small) model
# and its tokenizer using the  Transformers library, then puts the model
# in evaluation mode for inference.
# =====================================================================
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "google/flan-t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
model.eval()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [ ]:
# ==========================================================================
# This section allows the FastAPI app exposes a chat endpoint that first looks
# up a reply in a CSV via answer_from_csv, falls back to a generative model
# if no strong match exists, synthesizes the reply to MP3 with gTTS, and serves
# the audio from a static /audio path.
# ==========================================================================
from fastapi import FastAPI
from fastapi.responses import FileResponse
from fastapi.staticfiles import StaticFiles
from pydantic import BaseModel
import uvicorn, nest_asyncio, threading, time, uuid, os
from pyngrok import conf, ngrok
from gtts import gTTS

app = FastAPI()
app.mount("/audio", StaticFiles(directory="/content", html=True), name="audio")

class Prompt(BaseModel):
    prompt: str

@app.post("/respond")
async def respond(request: Prompt):
    user_msg = request.prompt.strip()
    csv_answer = answer_from_csv(user_msg, df, threshold=90)
    filename = f"audio_{uuid.uuid4().hex[:8]}.mp3"
    filepath = f"/content/{filename}"

    if csv_answer:
        gTTS(text=csv_answer, lang='en').save(filepath)
        return {"response": csv_answer, "audio_path": f"/audio/{filename}"}

    prompt_text = f"You are a helpful cultural assistant. Answer clearly.\nUser: {user_msg}\nAssistant:"
    input_ids = tokenizer(prompt_text, return_tensors="pt").input_ids
    with torch.no_grad():
        outputs = model.generate(input_ids, max_length=120)
    reply = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    gTTS(text=reply, lang='en').save(filepath)
    return {"response": reply, "audio_path": f"/audio/{filename}"}

@app.get("/audio/{filename}")
async def get_audio(filename: str):
    return FileResponse(path=f"/content/{filename}", media_type="audio/mpeg")


In [ ]:
# =====================================================================
# This snippet runs the FastAPI app with Uvicorn in a background thread
# (using nest_asyncio), opens an ngrok tunnel with the auth token, then
# builds and prints the public /respond endpoint (FASTAPI_URL) and the
# base URL for serving audio (AUDIO_BASE_URL).
# ======================================================================
def run_api():
    uvicorn.run(app, host="0.0.0.0", port=8000)

nest_asyncio.apply()
threading.Thread(target=run_api, daemon=True).start()
time.sleep(5)

conf.get_default().auth_token = "36Ig2CHEPLL7Ivf6J256Wgmd7i1_ThhuzTZLKDHhAEbosQCx"
public_url = ngrok.connect(8000)
FASTAPI_URL = f"{public_url.public_url}/respond"
AUDIO_BASE_URL = public_url.public_url
print("FASTAPI URL:", FASTAPI_URL)
print("AUDIO BASE:", AUDIO_BASE_URL)

INFO:     Started server process [29796]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


FASTAPI URL: https://unvibrated-tubercularly-darwin.ngrok-free.dev/respond
AUDIO BASE: https://unvibrated-tubercularly-darwin.ngrok-free.dev


In [ ]:
import os
os.environ["SMTP_HOST"]="smtp.gmail.com"
os.environ["SMTP_PORT"]="587"
os.environ["SMTP_USER"]=""
os.environ["SMTP_PASS"]="your_app_password"
os.environ["FROM_EMAIL"]=""

In [ ]:
# vet_finder_app.py — Cleaned, fixed and ready-to-run
# Features:
# - iMessage-like chat UI
# - Theme switching (Light/Dark)
# - Avatar picker (uses provided Imgur images)
# - Typing animation (simulated)
# - Whisper mic support (optional)
# - Sidebar toggle CSS fix
# - Robust session-state handling

import streamlit as st
import requests
import time
import html
import tempfile
import sys
from typing import Tuple, Optional

# Optional components
try:
    from streamlit_image_select import image_select
except Exception:
    image_select = None

# Optional whisper/mic
try:
    from streamlit_mic_recorder import mic_recorder
    import whisper
    WHISPER_OK = True
except Exception:
    WHISPER_OK = False

# Try to import FASTAPI_URL / AUDIO_BASE_URL from config.py created by ngrok cell
try:
    from config import FASTAPI_URL, AUDIO_BASE_URL
except Exception:
    FASTAPI_URL = ""
    AUDIO_BASE_URL = ""

# ---- Assets (use i.imgur direct links for embedding) ----
USER_AVATARS = [
    "https://i.imgur.com/z0TxKqZ.png",
    "https://i.imgur.com/olgbtrK.png",
    "https://i.imgur.com/86azvSc.png",
    "https://i.imgur.com/exSI77K.png",
]
BOT_AVATAR = "https://i.imgur.com/PCDvPM0.png"  # Dr.vetman

# ---- Page setup ----
st.set_page_config(page_title="Jamaica Vet Finder", page_icon="🐾", layout="wide")

# ---- Sidebar-fix CSS to ensure toggle works ----
st.markdown(
    """
    <style>
    /* Restore sidebar toggle and pointer events */
    button[kind="header"]{ visibility:visible !important; display:flex !important; }
    [data-testid="stSidebar"]{ pointer-events:auto !important; }
    [data-testid="stSidebarNav"]{ display:block !important; visibility:visible !important; }
    </style>
    """,
    unsafe_allow_html=True,
)

# ---- Base styling & iMessage-like UI ----
BASE_CSS = """
<style>
:root{--accent:#009B3A;--bg:#f5f7fa;--panel:#ffffff;--text:#1f2937}
body { background: linear-gradient(135deg,var(--bg), #c3cfe2); font-family: Inter, system-ui, -apple-system, 'Segoe UI', Roboto, 'Helvetica Neue', Arial; }
.block-container { padding: 18px 24px; }
.chat-wrapper { max-width: 960px; margin: 8px auto 40px auto; }
.chat-row { display:flex; gap:12px; margin:12px 0; align-items:flex-start; }
.chat-row.user { justify-content:flex-end; }
.bubble { max-width: 74%; padding:12px 16px; border-radius:16px; box-shadow: 0 6px 18px rgba(0,0,0,0.06); }
.bubble.user { background: linear-gradient(135deg,var(--accent), #00b341); color: white; border-bottom-right-radius:6px; }
.bubble.bot  { background: var(--panel); color: var(--text); border-bottom-left-radius:6px; }
.sender { font-size:11px; opacity:0.7; font-weight:700; margin-bottom:6px; text-transform:uppercase }
.msg-text { white-space: pre-wrap; line-height:1.45 }
.avatar { width:44px; height:44px; border-radius:10px; object-fit:cover }
.typing { font-style: italic; opacity:0.9 }
</style>
"""
st.markdown(BASE_CSS, unsafe_allow_html=True)

# ---- Utilities ----
def html_escape(s: str) -> str:
    return html.escape(s or "")


def get_bot_response(prompt: str, timeout: int = 30) -> Tuple[str, Optional[str]]:
    """Call backend FASTAPI. Expects FASTAPI_URL to be full endpoint (e.g. http://host:port/ask)
    Returns (text_response, audio_path_or_None)."""
    if not FASTAPI_URL:
        return ("API not configured. Re-run your ngrok cell to generate config.py.", None)
    try:
        res = requests.post(FASTAPI_URL, json={"prompt": prompt}, timeout=timeout)
        if res.status_code != 200:
            # try to extract json detail
            try:
                j = res.json()
                return (f"Server error {res.status_code}: {j.get('detail', j)}", None)
            except Exception:
                return (f"Server error {res.status_code}: {res.text}", None)
        data = res.json()
        return (data.get("response", "No response."), data.get("audio_path"))
    except Exception as e:
        return (f"Error contacting server: {e}", None)

# ---- Whisper STT helper (optional) ----
if WHISPER_OK:
    @st.cache_resource
    def _load_whisper_model():
        return whisper.load_model("tiny")

    def _stt_from_bytes(wav_bytes: bytes) -> str:
        try:
            with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp:
                tmp.write(wav_bytes)
                tmp.flush()
                model = _load_whisper_model()
                result = model.transcribe(tmp.name, language="en")
                return (result.get("text") or "").strip()
        except Exception as e:
            st.error(f"Speech-to-text failed: {e}")
            return ""
else:
    def _stt_from_bytes(_):
        return ""

# ---- Session state initialization ----
if "user_name" not in st.session_state:
    st.session_state.user_name = ""
if "avatar" not in st.session_state:
    st.session_state.avatar = USER_AVATARS[0]
if "avatar_chosen" not in st.session_state:
    st.session_state.avatar_chosen = False
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []  # list of tuples (sender, text)
if "show_feedback" not in st.session_state:
    st.session_state.show_feedback = False
if "star_rating" not in st.session_state:
    st.session_state.star_rating = 0
if "theme" not in st.session_state:
    st.session_state.theme = "Light"

# ---- Header and Theme control ----
col1, col2 = st.columns([3,1])
with col1:
    st.title("Jamaica Vet Finder 🐾🇯🇲")
    st.markdown("Find veterinary care for your pets across Jamaica — chat with Dr.vetman for help.")
with col2:
    theme = st.selectbox("Theme", ["Light","Dark"], index=0 if st.session_state.theme=="Light" else 1)
    if theme != st.session_state.theme:
        st.session_state.theme = theme
        st.experimental_rerun()

# Apply theme CSS variables
if st.session_state.theme == "Dark":
    st.markdown("<style>:root{--bg:#0b1220;--panel:#071028;--accent:#10b981;--text:#e6eef6}</style>", unsafe_allow_html=True)
else:
    st.markdown("<style>:root{--bg:#f5f7fa;--panel:#ffffff;--accent:#009B3A;--text:#1f2937}</style>", unsafe_allow_html=True)

# ---- Ask for name if missing ----
if not st.session_state.user_name:
    st.header("Quick setup")
    st.write("Tell me your name so Dr.vetman can greet you.")
    name = st.text_input("Your name", key="name_input")
    if name:
        st.session_state.user_name = name
        st.experimental_rerun()
    st.stop()

# ---- Avatar picker if not chosen ----
if not st.session_state.avatar_chosen:
    st.markdown("### 👤 Pick an avatar to start")
    if image_select is not None:
        sel = image_select("Choose avatar", USER_AVATARS, use_container_width=True)
        if sel:
            st.session_state.avatar = sel
    else:
        cols = st.columns(len(USER_AVATARS))
        for i, a in enumerate(USER_AVATARS):
            with cols[i]:
                if st.button(f"Avatar {i+1}", key=f"av_{i}"):
                    st.session_state.avatar = a
    if st.session_state.avatar:
        st.image(st.session_state.avatar, width=88)
    if st.button("Next"):
        if not st.session_state.avatar:
            st.warning("Please pick an avatar first")
        else:
            st.session_state.avatar_chosen = True
            st.experimental_rerun()
    st.stop()

# ---- Sidebar controls ----
with st.sidebar:
    st.markdown(f"### 👋 Hello {st.session_state.user_name}")
    st.markdown("---")
    st.markdown("**Chat Controls**")
    if st.button("Clear chat"):
        st.session_state.chat_history = []
        st.success("Cleared chat history")
    st.markdown("---")
    st.markdown(f"**Bot:** Dr.vetman")
    st.image(BOT_AVATAR, width=80)

# ---- Initial greeting if empty ----
if not any(m for m in st.session_state.chat_history if m and m[0] == "Dr.vetman"):
    greeting = ("Hello — I'm Dr.vetman. I can help locate vets, provide basic pet-care info, and suggest next steps.")
    st.session_state.chat_history.append(("Dr.vetman", greeting))

# ---- Chat rendering ----
chat_placeholder = st.container()
with chat_placeholder:
    st.markdown('<div class="chat-wrapper">', unsafe_allow_html=True)
    for i, (sender, raw_msg) in enumerate(st.session_state.chat_history):
        text_only = raw_msg if isinstance(raw_msg, str) else str(raw_msg)
        is_user = sender.startswith(st.session_state.user_name) or sender.startswith("You") or sender.startswith("Customer")
        role_class = "user" if is_user else "bot"
        align_class = "user" if is_user else "bot"
        avatar = st.session_state.avatar if is_user else BOT_AVATAR
        bubble_cls = "bubble user" if is_user else "bubble bot"

        # render
        html_block = f"<div class='chat-row {align_class}'><img class='avatar' src='{avatar}'/><div class='{bubble_cls}'><div class='sender'>{html_escape(sender)}</div><div class='msg-text'>{html_escape(text_only)}</div></div></div>"
        st.markdown(html_block, unsafe_allow_html=True)
    st.markdown('</div>', unsafe_allow_html=True)

# ---- Input row ----
input_col, send_col = st.columns([8,2])
with input_col:
    user_input = st.text_input("Type your question here:", key="user_input", placeholder="e.g. Where is the nearest vet in Kingston?")
with send_col:
    send_clicked = st.button("Send", use_container_width=True)

# ---- Whisper voice handling (if available) ----
if WHISPER_OK:
    voice = mic_recorder(start_prompt="🎤", stop_prompt="⏹️", just_once=True, format="wav", key="vincy_mic")
else:
    voice = None

if WHISPER_OK and voice and isinstance(voice, dict) and voice.get("bytes"):
    transcript = _stt_from_bytes(voice["bytes"]) if WHISPER_OK else ""
    if transcript:
        st.session_state.chat_history.append((st.session_state.user_name or "You", transcript))
        st.experimental_rerun()

# ---- Send flow ----
if send_clicked and st.session_state.user_input:
    val = st.session_state.user_input
    st.session_state.chat_history.append((st.session_state.user_name or "You", val))

    # show typing placeholder
    typing_ph = st.empty()
    with typing_ph.container():
        st.markdown(f"<div style='display:flex;gap:10px;align-items:center'><img class='avatar' src='{BOT_AVATAR}' width=44 height=44 /><div class='typing'>Dr.vetman is typing<span id='dots'>...</span></div></div>", unsafe_allow_html=True)
    # simulated typing delay
    time.sleep(1.2)

    reply, audio_path = get_bot_response(val)
    st.session_state.chat_history.append(("Dr.vetman", reply or "Sorry, I couldn't think of a reply."))
    typing_ph.empty()

    # attempt to play audio if available
    if audio_path and AUDIO_BASE_URL:
        try:
            full = f"{AUDIO_BASE_URL}{audio_path}"
            audio_bytes = requests.get(full, timeout=30).content
            st.audio(audio_bytes, format='audio/mp3')
        except Exception as e:
            st.warning(f"Failed to load TTS audio: {e}")

    # clear input and rerun to show message
    st.session_state.user_input = ""
    st.experimental_rerun()

# ---- Rating / feedback ----
st.markdown("---")
if st.button("End Conversation"):
    st.session_state.show_feedback = True

if st.session_state.get("show_feedback"):
    st.markdown("### ⭐ Rate your experience")
    rating = st.slider("How would you rate Dr.vetman?", 1, 5, 5)
    feedback = st.text_area("Any feedback? (optional)")
    if st.button("Submit Feedback"):
        st.session_state.star_rating = rating
        # TODO: send/store feedback to backend
        st.success("Thanks — feedback recorded!")

# ---- Footer / tips ----
st.markdown("---")
st.markdown("*Tip: If FASTAPI_URL/AUDIO_BASE_URL are empty, re-run your ngrok cell to create config.py.*")


2025-12-03 01:32:11.514 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-03 01:32:11.515 No runtime found, using MemoryCacheStorageManager
2025-12-03 01:32:11.518 No runtime found, using MemoryCacheStorageManager
2025-12-03 01:32:11.585 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-03 01:32:12.745 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-03 01:32:12.749 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-03 01:32:12.987 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-12-03 01:32:12.988 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-03 01:32:12.989 Thread 'MainThread': missing ScriptRunContext! This warning can be

DeltaGenerator()

In [ ]:
import time

# Kill Streamlit
!pkill -f streamlit
!fuser -k 8501/tcp || true
time.sleep(3)

# Start fresh
!streamlit run vet_finder_app.py --server.headless true --server.port 8501 > /content/streamlit_log.txt 2>&1 &
time.sleep(10)

print("✅ Restarted! Refresh your browser")

8501/tcp:            30993
✅ Restarted! Refresh your browser


In [ ]:
import time
import os

# Kill any existing Streamlit process on 8501
os.system("fuser -k 8501/tcp || true")

# Wait a second
time.sleep(2)

# Make sure your updated code is saved
# e.g., write your latest code to vet_finder_app.py
# Path("vet_finder_app.py").write_text(latest_code_string)  # if using string

# Restart Streamlit in the background
os.system("nohup streamlit run vet_finder_app.py --server.headless true --server.port 8501 > streamlit_log.txt 2>&1 &")

# Wait for Streamlit to start
time.sleep(8)

# Print message
print("Streamlit restarted! Refresh your browser at:", streamlit_url)




NameError: name 'streamlit_url' is not defined